<a href="https://colab.research.google.com/github/netrapathak/dataria-capstone/blob/master/Clean_vertical_to_patent_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import os
from cleanco import cleanco
import re
import unicodedata
!pip install ftfy
from ftfy import fix_text
import sys
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
!pip install sparse_dot_topn 
import sparse_dot_topn.sparse_dot_topn as ct
!pip install mysql-connector-python
import mysql.connector

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [0]:
#USPTO Data. No need to change.
uspto = pd.read_csv('uspto.csv', dtype='unicode')
#This stores the emerging spaces csv data. Change this to find matches with USPTO data.
ai_vert = pd.read_csv('ai_drug_disc_vert.csv', dtype='unicode')

In [0]:
#Run this
def ngrams(string, n=3):
    string = fix_text(string) 
    string = string.encode("ascii", errors="ignore").decode() 
    string = string.lower() 
    chars_to_remove = [")","(",".","|","[","]","{","}","'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string) 
    string = string.replace('&', 'and')
    string = string.replace(',', ' ')
    string = string.replace('-', ' ')
    string = string.title() 
    string = re.sub(' +',' ',string).strip()
    string = ' '+ string +' ' 
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [0]:
#Run this.
company_names = uspto['organization']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(company_names.values.astype('U'))

In [0]:
#Change column name or df name as required for each emerging space data. 
company = ai_vert['Company Legal Name']
tf_idf_matrix_alt = vectorizer.fit_transform(company.values.astype('U'))

In [0]:
(ai_vert[ai_vert['Company Name'] == 'Neon Therapeutics'])

In [0]:
#Don't run this
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [0]:
#Don't run this
import time
t1 = time.time()
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.8)
t = time.time()-t1
print("SELFTIMED:", t)

In [0]:
#Don't Run this
def get_matches_df(sparse_matrix, name_vector, top=1000):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [0]:
#Don't Run this
matches_df = get_matches_df(matches, company)
matches_df = matches_df[matches_df['similairity'] > 0.80] # Remove all exact matches
matches_df.sample(50)

In [0]:
#Run this
org_name_clean = ai_vert['Company Legal Name']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
tfidf = vectorizer.fit_transform(org_name_clean.astype('U'))
print('Vecorizing completed...')
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tfidf)
org_column = uspto['organization'] #column to match against in the messy data

unique_org = set(org_column.astype('U')) 

def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

import time
t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(unique_org)
t = time.time()-t1
print("COMPLETED IN:", t)
unique_org = list(unique_org) 


Vecorizing completed...
getting nearest n...
COMPLETED IN: 34.626405000686646


In [0]:
#Run this
matches = []
for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), ai_vert.values[j][0][1],unique_org[i]]
    matches.append(temp)
print('Building data frame...')  
matches = pd.DataFrame(matches, columns=['Match confidence (lower is better)','Matched name','Original name'])
print('Done')

Building data frame...
Done


In [0]:
#Run this
match = matches.sort_values(by=['Match confidence (lower is better)'])

In [0]:
#Run this
match_1 = match[match['Match confidence (lower is better)'] < 0.41]

match_1

In [0]:
#Run this. The column 'Matched name' is pitchbook company name. 'Original name' is from USPTO. use 'Original name' for 
# MySQL queries. 
match_1

,Match confidence (lower is better),Matched name,Original name
373445,0.00,Globavir Biosciences,"GLOBAVIR BIOSCIENCES, INC."
77094,0.00,Perthera,"PERTHERA, INC."
420812,0.00,Evoke Neuroscience,"Evoke Neuroscience, Inc."
384212,0.00,SOM Biotech,"SOM INNOVATION BIOTECH, S.L."
151745,0.00,Numerate,"Numerate, Inc."
369305,0.00,Atomwise,ATOMWISE INC.
366295,0.00,Kyndi,"KYNDI, INC."
171352,0.00,ThoughtSpot,"THOUGHTSPOT, INC."
316091,0.00,Berkeley Lights,"Berkeley Lights, Inc."
67619,0.00,Arzeda,ARZEDA CORPORATION


In [0]:
#Run this to establish a connection with USPTO database. 
connection = mysql.connector.connect(host='acrs-data-sources.cagyz4rarynl.us-west-2.rds.amazonaws.com',
                                         database='PatentsView_20181127',
                                         user='uwmis',
                                         password='patents2020')




In [0]:

cursor = connection.cursor()
li = []
for row in comp_list:
    print(row)
    #If you need more detail, make changes to the query listed below
    Query = ("select p.patent_id , a.organization , p.year from patent p, assignee a, temp_patent_firstnamed_assignee temp where temp.patent_id = p.patent_id and temp.assignee_id = a.assignee_id and a.organization = '{}'".format(row))
    cursor.execute(Query)
    records = cursor.fetchall()
    li.append(records)
    
    

GLOBAVIR BIOSCIENCES, INC.
PERTHERA, INC.
Evoke Neuroscience, Inc.
SOM INNOVATION BIOTECH, S.L.
Numerate, Inc.
ATOMWISE INC.
KYNDI, INC.
THOUGHTSPOT, INC.
Berkeley Lights, Inc.
ARZEDA CORPORATION
FDNA INC.
MOLECULAR HEALTH GMBH
uBiome, Inc.
Aetion, Inc.
nan
Gritstone Oncology, Inc.
Accutar Biotechnology Inc.
E-Therapeutics PLC
Biodesix, Inc.
BIOVISTA, INC.
MERIDIGEN BIOTECH CO., LTD.
Berg LLC
EVOKE NEUROSCIENCE
Immutopics, Inc.


In [0]:
#List is not flat. this flattens the list. 
flat_list = [item for sublist in li for item in sublist]

In [0]:
#Converting list to dataframe.
patent_companies = pd.DataFrame(flat_list,columns=['Patent ID', 'Company Name', 'Patent Year'])

In [0]:
patent_companies

,Patent ID,Company Name,Patent Year
0,9321749,"GLOBAVIR BIOSCIENCES, INC.",2016
1,9669038,"GLOBAVIR BIOSCIENCES, INC.",2017
2,D826972,"PERTHERA, INC.",2018
3,D827664,"PERTHERA, INC.",2018
4,8380316,"Evoke Neuroscience, Inc.",2013
...,...,...,...
109,9901637,Berg LLC,2018
110,9165472,EVOKE NEUROSCIENCE,2015
111,6838264,"Immutopics, Inc.",2005
112,7670805,"Immutopics, Inc.",2010


In [0]:
patent_companies.to_csv('ai_drug_disc_vert_patents.csv')